In [3]:
import os

In [4]:
os.chdir('../')

In [5]:
%pwd

'c:\\Users\\PC\\Documents\\projects\\Power_Consumption_In_Tetouan'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [7]:
from powerconsumptiontetouan.constants import *
from powerconsumptiontetouan.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

<!-- df.Datetime = pd.to_datetime(df.Datetime)
print(f'The datatype for Datetime is: {df.Datetime.dtype}.\n')

df['month'] = df.Datetime.dt.month
df['weekday'] = df.Datetime.dt.day
df['time'] = df.Datetime.dt.time.astype(str) -->

In [13]:
import os
from powerconsumptiontetouan import logger
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import pandas as pd

In [10]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config



    def preprocessing_features_data(self):
        data = pd.read_csv(self.config.data_path)
        data.Datetime = pd.to_datetime(data.Datetime)
        data['month'] = data.Datetime.dt.month
        data['weekday'] = data.Datetime.dt.day
        data['time'] = data.Datetime.dt.time.astype(str)
        features = ['Temperature', 'Humidity', 'WindSpeed', 'GeneralDiffuseFlows', 'DiffuseFlows',  'month', 'weekday', 'time']
        targets = ['PowerConsumption_Zone1','PowerConsumption_Zone2', 'PowerConsumption_Zone3']
        data_features =data[features]
        numerical_features = data_features.drop(columns=['time']).columns
        time_feature = ['time']
        scaler = MinMaxScaler()
        data_features[numerical_features] = scaler.fit_transform(data_features[numerical_features])
        encoder = OneHotEncoder(handle_unknown="ignore")
        # OneHot encode the 'time' feature
        time_encoded = encoder.fit_transform(data_features[time_feature])

        time_encoded_df = pd.DataFrame(time_encoded.todense())
        # Concatenate the encoded time values with the scaled DataFrame

        data_features = pd.concat([data_features, time_encoded_df], axis=1)

        # Drop the original 'time' and 'time_seconds' columns
        data_features = data_features.drop(columns=['time'])

        data_targets=data[targets]

        data_ml=pd.concat([data_features,data_targets],axis=1)

        train, test = train_test_split(data_ml,test_size=0.33)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)

    
    


In [12]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.preprocessing_features_data()
except Exception as e:
    raise e

[2024-07-30 02:56:35,436: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-30 02:56:35,437: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-30 02:56:35,438: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-30 02:56:35,439: INFO: common: created directory at: artifacts]
[2024-07-30 02:56:35,440: INFO: common: created directory at: artifacts/data_transformation]


C:\Users\PC\AppData\Local\Temp\ipykernel_7492\2267528994.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_features[numerical_features] = scaler.fit_transform(data_features[numerical_features])
